In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')
os.getcwd()

'/home/yermaand/audio-nn'

In [3]:
from src import *
import pandas as pd
from natsort import natsorted, index_natsorted

In [4]:
files = load_files_from_dataset('002_eyedea_RX100')

In [5]:
table = pd.read_csv('data/annotations/RX100_vehicle_count_manual_annotation_with_sound.csv')

In [6]:
summary_table = [] 
for file in files:
    subtable = table[table.iloc[:, 0].str.contains(file)]
    indices = index_natsorted(subtable['name'])
    subtable = subtable.to_numpy()[indices]
    manual_annotations = subtable[:, 1].astype(int) + subtable[:, 2].astype(int)    
    automatic_annotations = pd.read_csv(f'data/manual_annotation_RX100/{file}.csv').to_numpy()[:, 1]

    diff = automatic_annotations.sum() - manual_annotations.sum()
    rvce = np.abs(diff) / manual_annotations.sum()

    # print(file, rvce)
    summary_table.append([file, rvce, automatic_annotations.sum(), manual_annotations.sum(), diff])

    # np.savetxt(f'data/manual_counts/{file}.txt', [manual_annotations.sum()], fmt='%s')

# print(tabulate(summary_table, headers=['file', 'rvce', 'manual', 'automatic'], tablefmt='fancy_grid'))

#! footer
summary_table.append([
    'total',
    np.mean([x[1] for x in summary_table]),
    np.mean([x[2] for x in summary_table]),
    np.mean([x[3] for x in summary_table]),
    np.mean([x[4] for x in summary_table]),
])

pd.DataFrame(summary_table, columns=['file', 'rvce', 'automatic', 'manual', 'diff'])

,file,rvce,automatic,manual,diff
0,05_RX100,0.000000,178.000000,178.000000,0.00
1,06_RX100,0.026316,185.000000,190.000000,-5.00
2,07_RX100,0.253425,109.000000,146.000000,-37.00
3,09_RX100,0.024691,79.000000,81.000000,-2.00
4,10_RX100,0.031674,214.000000,221.000000,-7.00
5,11_RX100,0.004651,214.000000,215.000000,-1.00
6,12_RX100,0.068182,41.000000,44.000000,-3.00
7,13_RX100,0.017857,110.000000,112.000000,-2.00
8,14_RX100,0.155340,87.000000,103.000000,-16.00
9,15_RX100,0.010363,195.000000,193.000000,2.00


In [7]:
pd.DataFrame(summary_table, columns=['file', 'rvce: n_counts', 'automatic', 'manual', 'diff']).to_csv('outputs/000_manual/tst_rvce_counts.csv')

In [8]:
files_RX100 = load_files_from_dataset('005_eyedea_RX100_smaller')
files_iphone = load_files_from_dataset('006_eyedea_iphone_smaller')

In [21]:
summary_table = []

for file_RX100, file_iphone in zip(files_RX100, files_iphone):
    events = load_events(file_iphone)
    audio_RX100, sr = load_audio(file_RX100, return_sr=True)
    audio, sr = load_audio(file_iphone, return_sr=True)
    
    audio_length_RX100 = get_signal_length(audio_RX100, sr)
    audio_length = get_signal_length(audio, sr)
    
    subtable = table[table.iloc[:, 0].str.contains(file_RX100)]
    indices = index_natsorted(subtable['name'])
    subtable = subtable.to_numpy()[indices]
    
    # print(audio_length - audio_length_RX100)
    diff = abs(audio_length - audio_length_RX100)
    if diff > 6:
        print(diff)
        print(file_iphone)
        print()
        continue

    manual_annotations = subtable[:, 1].astype(int) + subtable[:, 2].astype(int)    
    n_manual = manual_annotations.sum()
    n_automatic = len(load_events(file_iphone))

    diff = n_automatic - n_manual    
    rvce = np.abs(diff) / n_manual

    summary_table.append([file_iphone, rvce, n_automatic, n_manual, diff])

#! footer
summary_table.append([
    'total',
    np.mean([x[1] for x in summary_table]),
    np.mean([x[2] for x in summary_table]),
    np.mean([x[3] for x in summary_table]),
    np.mean([x[4] for x in summary_table]),
])

pd_summary = pd.DataFrame(summary_table, columns=['file', 'rvce: n_counts', 'automatic', 'manual', 'diff'])
pd_summary.to_csv('outputs/000_manual/tst_iphone_rvce_output.csv')

486
18_Prazska_iphone



In [31]:
def prepare_table(files_reference, files_targets):
    summary_table = []

    for file_reference, file_target in zip(files_reference, files_targets):
        print(file_reference, file_target)
        audio_RX100, sr = load_audio(file_reference, return_sr=True)
        audio, sr = load_audio(file_target, return_sr=True)
        
        audio_length_RX100 = get_signal_length(audio_RX100, sr)
        audio_length = get_signal_length(audio, sr)
        
        # print(audio_length - audio_length_RX100)
        diff = abs(audio_length - audio_length_RX100)
        if diff > 10:
            print('Signal length difference:', diff)
            print('-' * 30)
            continue

        subtable = table[table.iloc[:, 0].str.contains(file_reference)]
        indices = index_natsorted(subtable['name'])
        subtable = subtable.to_numpy()[indices]

        manual_annotations = subtable[:, 1].astype(int) + subtable[:, 2].astype(int)    
        n_manual = manual_annotations.sum()
        n_automatic = len(load_events(file_target))

        diff = n_automatic - n_manual    
        rvce = np.abs(diff) / n_manual

        summary_table.append([file_target, rvce, n_automatic, n_manual, diff])

    #! footer
    summary_table.append([
        'total',
        np.mean([x[1] for x in summary_table]),
        np.mean([x[2] for x in summary_table]),
        np.mean([x[3] for x in summary_table]),
        np.mean([x[4] for x in summary_table]),
    ])

    return summary_table

In [28]:
files_reference = load_files_from_dataset('005_eyedea_RX100_smaller')
files_targets = load_files_from_dataset('006_eyedea_iphone_smaller')

summary_table = prepare_table(files_reference, files_targets)
pd_summary = pd.DataFrame(summary_table, columns=['file', 'rvce: n_counts', 'automatic', 'manual', 'diff'])
pd_summary.to_csv('outputs/000_manual/tst_iphone_rvce_output.csv')

05_RX100 05_iphone
09_RX100 09_iphone
10_RX100 10_iphone
11_RX100 11_iphone
12_RX100 12_iphone
13_RX100 13_iphone
15_RX100 15_iphone
16_RX100 16_iphone
17_RX100 17_iphone
18_Prazska_RX100 18_Prazska_iphone
Signal length difference: 486
----------
19_Krocehlavska_RX100 19_Krocehlavska_iphone
20_Kovary_RX100 20_Kovary_iphone
21_Videnska_RX100 21_Videnska_iphone
22_Jesenicka_RX100 22_Jesenicka_iphone
23_KeZlatnikum_RX100 23_KeZlatnikum_iphone
24_Cernokostelecka_RX100 24_Cernokostelecka_iphone
25_Hlavni_RX100 25_Hlavni_iphone
26_Novosibrinska_RX100 26_Novosibrinska_iphone
27_Komoranska_RX100 27_Komoranska_iphone
28_BrezanskeUdoli_RX100 28_BrezanskeUdoli_iphone
29_KPrehradam_RX100 29_KPrehradam_iphone
30_RadotinskaS_RX100 30_RadotinskaS_iphone
31_RadotinskaJ2_RX100 31_RadotinskaJ2_iphone
32_Cementarska_RX100 32_Cementarska_iphone
33_Vypadova_RX100 33_Vypadova_iphone
34_PrazskaDobrichovice_RX100 34_PrazskaDobrichovice_iphone
35_Rovinska_RX100 35_Rovinska_iphone
36_Jesenicka_RX100 36_Jesenick

In [32]:
files_reference = load_files_from_dataset('005_eyedea_RX100_smaller')
files_targets = load_files_from_dataset('006_eyedea_Mobius_smaller')

summary_table = prepare_table(files_reference, files_targets)
pd_summary = pd.DataFrame(summary_table, columns=['file', 'rvce: n_counts', 'automatic', 'manual', 'diff'])
pd_summary.to_csv('outputs/000_manual/tst_Mobius_rvce_output.csv')

05_RX100 05_Mobius
09_RX100 09_Mobius
10_RX100 10_Mobius
11_RX100 11_Mobius
12_RX100 12_Mobius
13_RX100 13_Mobius
15_RX100 15_Mobius
16_RX100 16_Mobius
17_RX100 17_Mobius
18_Prazska_RX100 18_Prazska_Mobius
19_Krocehlavska_RX100 19_Krocehlavska_Mobius
20_Kovary_RX100 20_Kovary_Mobius
21_Videnska_RX100 21_Videnska_Mobius
22_Jesenicka_RX100 22_Jesenicka_Mobius
23_KeZlatnikum_RX100 23_KeZlatnikum_Mobius
Signal length difference: 819
------------------------------
24_Cernokostelecka_RX100 24_Cernokostelecka_Mobius
25_Hlavni_RX100 25_Hlavni_Mobius
26_Novosibrinska_RX100 26_Novosibrinska_Mobius
27_Komoranska_RX100 27_Komoranska_Mobius
28_BrezanskeUdoli_RX100 28_BrezanskeUdoli_Mobius
29_KPrehradam_RX100 29_KPrehradam_Mobius
30_RadotinskaS_RX100 30_RadotinskaS_Mobius
31_RadotinskaJ2_RX100 31_RadotinskaJ2_Mobius
32_Cementarska_RX100 32_Cementarska_Mobius
33_Vypadova_RX100 33_Vypadova_Mobius
34_PrazskaDobrichovice_RX100 34_PrazskaDobrichovice_Mobius
35_Rovinska_RX100 35_Rovinska_Mobius
36_Jesenic